# Singapore Resale Flat Price Prediction

## Data collection

In [1]:
import pandas as pd
import glob
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score


In [2]:
df1 = pd.concat([pd.read_csv(x) for x in glob.glob("*.csv")], ignore_index= True)
df1.shape

(917996, 11)

## Handling missing values

In [3]:
df1.isnull().sum()

month                       0
town                        0
flat_type                   0
block                       0
street_name                 0
storey_range                0
floor_area_sqm              0
flat_model                  0
lease_commence_date         0
resale_price                0
remaining_lease        709050
dtype: int64

In [4]:
df1.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,NaN
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0,NaN
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0,NaN
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0,NaN
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0,NaN


In [5]:


(df1.remaining_lease.isnull().sum() / df1.shape[0]) * 100



77.23889864443854

In [6]:
### Almost 77.25 percentage of the remaining lease column is Nan. If I do Dropna 
### it the data will be lease. Hence, decided to drop the remaining lease columns for better ML model

### Also dropping column month and block.

df2 = df1.drop(columns=["remaining_lease", "month", "block"])
df2.shape

(917996, 8)

In [7]:
df2.head()

,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,ANG MO KIO,3 ROOM,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0
1,ANG MO KIO,3 ROOM,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0
2,ANG MO KIO,3 ROOM,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0
3,ANG MO KIO,3 ROOM,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0
4,ANG MO KIO,3 ROOM,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0


In [8]:
df2.flat_type.value_counts()

flat_type
4 ROOM              348459
3 ROOM              293704
5 ROOM              193333
EXECUTIVE            69092
2 ROOM               11569
1 ROOM                1300
MULTI GENERATION       279
MULTI-GENERATION       260
Name: count, dtype: int64

### Feature Engineering

In [9]:
def flat_type(x):
    if x in ("EXECUTIVE", "MULTI GENERATION", "MULTI-GENERATION"):
        return int(6)
    else:
        token = x.split(" ")
        return int(token[0])
    
    

In [10]:
### Converting the values of the flat_type feature into the numeric value

df2["flat_type"] = df2["flat_type"].apply(flat_type)


In [11]:
df2.flat_type.value_counts()

flat_type
4    348459
3    293704
5    193333
6     69631
2     11569
1      1300
Name: count, dtype: int64

In [12]:
def storey_range_avg(x):
    token = x.split(" ")
    avg = ( int(token[0]) + int(token[2]) ) / 2
    return avg

In [13]:
### Replacing the values of storey_range feature with the average floors

df2["storey_range"] = df2["storey_range"].apply(storey_range_avg)

In [14]:
df2.head()

,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price
0,ANG MO KIO,3,ANG MO KIO AVE 4,8.0,69.0,Improved,1986,147000.0
1,ANG MO KIO,3,ANG MO KIO AVE 4,5.0,61.0,Improved,1986,144000.0
2,ANG MO KIO,3,ANG MO KIO AVE 1,8.0,73.0,New Generation,1976,159000.0
3,ANG MO KIO,3,ANG MO KIO AVE 1,8.0,73.0,New Generation,1976,167000.0
4,ANG MO KIO,3,ANG MO KIO AVE 1,8.0,67.0,New Generation,1976,163000.0


In [15]:
def remaning_lease(year):
    current_year = datetime.now().year
    rem = current_year - year
    return rem

In [16]:
df2["remaining_lease"] = df2["lease_commence_date"].apply(remaning_lease)

In [17]:
df2.head()

,town,flat_type,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,ANG MO KIO,3,ANG MO KIO AVE 4,8.0,69.0,Improved,1986,147000.0,38
1,ANG MO KIO,3,ANG MO KIO AVE 4,5.0,61.0,Improved,1986,144000.0,38
2,ANG MO KIO,3,ANG MO KIO AVE 1,8.0,73.0,New Generation,1976,159000.0,48
3,ANG MO KIO,3,ANG MO KIO AVE 1,8.0,73.0,New Generation,1976,167000.0,48
4,ANG MO KIO,3,ANG MO KIO AVE 1,8.0,67.0,New Generation,1976,163000.0,48


In [18]:
### As we created feature with remaining lease we can drop the feature lease_commence_date

df3 = df2.drop(columns=["lease_commence_date", "street_name"])

df3.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,resale_price,remaining_lease
0,ANG MO KIO,3,8.0,69.0,Improved,147000.0,38
1,ANG MO KIO,3,5.0,61.0,Improved,144000.0,38
2,ANG MO KIO,3,8.0,73.0,New Generation,159000.0,48
3,ANG MO KIO,3,8.0,73.0,New Generation,167000.0,48
4,ANG MO KIO,3,8.0,67.0,New Generation,163000.0,48


In [19]:
df3["town"].unique()

array(['ANG MO KIO', 'BEDOK', 'BISHAN', 'BUKIT BATOK', 'BUKIT MERAH',
       'BUKIT PANJANG', 'BUKIT TIMAH', 'CENTRAL AREA', 'CHOA CHU KANG',
       'CLEMENTI', 'GEYLANG', 'HOUGANG', 'JURONG EAST', 'JURONG WEST',
       'KALLANG/WHAMPOA', 'MARINE PARADE', 'PASIR RIS', 'QUEENSTOWN',
       'SENGKANG', 'SERANGOON', 'TAMPINES', 'TOA PAYOH', 'WOODLANDS',
       'YISHUN', 'SEMBAWANG', 'PUNGGOL', 'LIM CHU KANG'], dtype=object)

In [50]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
label_encoder = LabelEncoder()


# Fit label encoder on both "flat_model" and "town" features
label_encoder.fit(pd.concat([df3["flat_model"], df3["town"]]))

# Transform the "flat_model" feature
df3["flat_model_encoded"] = label_encoder.transform(df3["flat_model"])

# Transform the "town" feature
df3["town_encoded"] = label_encoder.transform(df3["town"])

# Display the first few rows of the DataFrame with the encoded features
print(df3[df3["town"] == "BEDOK"])


         town  flat_type  storey_range  floor_area_sqm  flat_model  \
157     BEDOK          2          11.0            45.0    Improved   
158     BEDOK          2           2.0            45.0    Improved   
159     BEDOK          2           8.0            45.0    Improved   
160     BEDOK          2           8.0            45.0    Improved   
161     BEDOK          3           2.0            59.0    Improved   
...       ...        ...           ...             ...         ...   
915601  BEDOK          6          11.0           148.0  MAISONETTE   
915602  BEDOK          6          11.0           148.0  MAISONETTE   
915603  BEDOK          6          11.0           152.0  MAISONETTE   
915604  BEDOK          6          11.0           146.0  MAISONETTE   
915605  BEDOK          6           2.0           143.0  MAISONETTE   

        resale_price  remaining_lease  town_encoded  flat_model_encoded  
157         120000.0               38             7                  21  
158        

In [46]:
import pickle
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [43]:
df3['flat_model'].unique()

array(['Improved', 'New Generation', 'Model A', 'Standard', 'Apartment',
       'Simplified', 'Model A-Maisonette', 'Maisonette',
       'Multi Generation', 'Adjoined flat', 'Premium Apartment',
       'Terrace', 'Improved-Maisonette', 'Premium Maisonette', '2-room',
       'Model A2', 'Type S1', 'Type S2', 'DBSS', 'Premium Apartment Loft',
       '3Gen', 'IMPROVED', 'NEW GENERATION', 'MODEL A', 'STANDARD',
       'SIMPLIFIED', 'MODEL A-MAISONETTE', 'APARTMENT', 'MAISONETTE',
       'TERRACE', '2-ROOM', 'IMPROVED-MAISONETTE', 'MULTI GENERATION',
       'PREMIUM APARTMENT'], dtype=object)

In [23]:
df4 = df3.copy()

In [24]:
df4.drop(columns=["town", "flat_model"], inplace=True)
df4.head()

,flat_type,storey_range,floor_area_sqm,resale_price,remaining_lease,town_encoded,flat_model_encoded
0,3,8.0,69.0,147000.0,38,0,9
1,3,5.0,61.0,144000.0,38,0,9
2,3,8.0,73.0,159000.0,48,0,21
3,3,8.0,73.0,167000.0,48,0,21
4,3,8.0,67.0,163000.0,48,0,21


In [25]:
X = df4.drop(columns=['resale_price'], axis='columns')
X.head()

,flat_type,storey_range,floor_area_sqm,remaining_lease,town_encoded,flat_model_encoded
0,3,8.0,69.0,38,0,9
1,3,5.0,61.0,38,0,9
2,3,8.0,73.0,48,0,21
3,3,8.0,73.0,48,0,21
4,3,8.0,67.0,48,0,21


In [26]:
X.shape

(917996, 6)

In [27]:
y = df4["resale_price"]
y.head()

0    147000.0
1    144000.0
2    159000.0
3    167000.0
4    163000.0
Name: resale_price, dtype: float64

In [28]:
y.shape

(917996,)

### Model building

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [30]:
X_train.head(), y_train.head()

(        flat_type  storey_range  floor_area_sqm  remaining_lease  \
 735961          4           2.0           104.0               36   
 69642           5           8.0           121.0               27   
 337857          6          14.0           130.0               23   
 576187          6          11.0           146.0               31   
 337410          4           2.0            92.0               46   
 
         town_encoded  flat_model_encoded  
 735961            26                  12  
 69642             25                   9  
 337857             5                  23  
 576187             1                   5  
 337410             1                  21  ,
 735961    180000.0
 69642     303000.0
 337857    562000.0
 576187    985000.0
 337410    380000.0
 Name: resale_price, dtype: float64)

In [31]:
### checking accuracy in Decision Tree
from sklearn.metrics import r2_score
dt_reg = DecisionTreeRegressor().fit(X_train, y_train)

y_pred_train = dt_reg.predict(X_train)
y_pred_test = dt_reg.predict(X_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

accuracy_metrics = {'Algorithm': "DecisionTreeRegressor",
                    'R2_train' : r2_train,
                        'R2_test'  : r2_test}

print(accuracy_metrics)

{'Algorithm': 'DecisionTreeRegressor', 'R2_train': 0.7405216240459528, 'R2_test': 0.682879992143925}


In [32]:
### Checking accuracy in Linear Regression

ln_reg = LinearRegression().fit(X_train, y_train)

y_pred_train = ln_reg.predict(X_train)
y_pred_test = ln_reg.predict(X_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

accuracy_metrics = {'Algorithm': "LinearRegression",
                    'R2_train' : r2_train,
                        'R2_test'  : r2_test}

print(accuracy_metrics)

{'Algorithm': 'LinearRegression', 'R2_train': 0.4839248646829495, 'R2_test': 0.4840714795022204}


In [33]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 734396 entries, 735961 to 345353
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   flat_type           734396 non-null  int64  
 1   storey_range        734396 non-null  float64
 2   floor_area_sqm      734396 non-null  float64
 3   remaining_lease     734396 non-null  int64  
 4   town_encoded        734396 non-null  int64  
 5   flat_model_encoded  734396 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 39.2 MB


In [34]:
### checking with ExtraTreeRegressor
from sklearn.metrics import r2_score
et_reg = ExtraTreeRegressor().fit(X_train, y_train)

y_pred_train = et_reg.predict(X_train)
y_pred_test = et_reg.predict(X_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

accuracy_metrics = {'Algorithm': "ExtraTreeRegressor",
                    'R2_train' : r2_train,
                        'R2_test'  : r2_test}

print(accuracy_metrics)




{'Algorithm': 'ExtraTreeRegressor', 'R2_train': 0.7405216240459528, 'R2_test': 0.6824254363438742}


In [35]:
from sklearn.metrics import mean_squared_error
from math import sqrt

mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)

rmse_train = sqrt(mse_train)
rmse_test = sqrt(mse_test)

print("RMSE for training set:", rmse_train)
print("RMSE for testing set:", rmse_test)

RMSE for training set: 85622.63880895691
RMSE for testing set: 94726.07648956952


In [36]:
### checking with RandomForestRegressor

model_reg = RandomForestRegressor().fit(X_train, y_train)

y_pred_train = model_reg.predict(X_train)
y_pred_test = model_reg.predict(X_test)
r2_train = r2_score(y_train, y_pred_train)
r2_test = r2_score(y_test, y_pred_test)

accuracy_metrics = {'Algorithm': "RandomForestRegressor",
                    'R2_train' : r2_train,
                        'R2_test'  : r2_test}

print(accuracy_metrics)

{'Algorithm': 'RandomForestRegressor', 'R2_train': 0.7393248952626152, 'R2_test': 0.6915500986197737}


In [37]:
X_train.head(), y_train.head()

(        flat_type  storey_range  floor_area_sqm  remaining_lease  \
 735961          4           2.0           104.0               36   
 69642           5           8.0           121.0               27   
 337857          6          14.0           130.0               23   
 576187          6          11.0           146.0               31   
 337410          4           2.0            92.0               46   
 
         town_encoded  flat_model_encoded  
 735961            26                  12  
 69642             25                   9  
 337857             5                  23  
 576187             1                   5  
 337410             1                  21  ,
 735961    180000.0
 69642     303000.0
 337857    562000.0
 576187    985000.0
 337410    380000.0
 Name: resale_price, dtype: float64)

In [38]:
features_values = [[4, 2.0, 104.0, 36, 26, 12]] 

prediction = dt_reg.predict(features_values)

# Display the prediction
print("Predicted resale price:", prediction)

Predicted resale price: [238812.94488189]


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


In [39]:
import pickle

with open('resale_flat_model.pkl', 'wb') as f:
    pickle.dump(dt_reg, f)